In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree
from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
from math import isnan

In [2]:
df = pd.read_csv("../Datasets/pass_types.csv")
df2 = pd.read_csv("../Datasets/passing_stats.csv")

### About the dataset (pass_types.csv) df
- **Nation:** 
  - Records in international play at senior level
  - Records in international play at youth level
  - Citizenship presented on Wikipedia
  - Birthplace when available

- **Pos:** Position most commonly played:
  - GK: Goalkeepers
  - DF: Defenders (FB, LB, RB, CB)
  - MF: Midfielders (DM, CM, LM, RM, WM, LW, RW, AM)
  - FW: Forwards

- **Age:** Age at the start of the season

- **90s:** 90s played (minutes played divided by 90)

- **Passing Statistics:**
  - Passes Attempted (Att) by type:
    - Live-ball Passes
    - Dead-ball Passes (includes free kicks, corner kicks, kickoffs, throw-ins, and goal kicks)
    - Passes from Free Kicks (FK)
    - Through Balls (TB)
    - Switches (Sw)
    - Crosses (Crs)
    - Throw-ins Taken (TI)
    - Corner Kicks (CK) with subcategories:
      - Inswinging (In)
      - Outswinging (Out)
      - Straight (Str)

- **Outcomes of Passes:**
  - Passes Completed (Cmp)
  - Offsides (Off)
  - Passes Blocked (Blocks)

In [3]:
df.head()

,Unnamed: 0,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Pass Types,Pass Types.1,Pass Types.2,...,Pass Types.5,Pass Types.6,Pass Types.7,Corner Kicks,Corner Kicks.1,Corner Kicks.2,Outcomes,Outcomes.1,Outcomes.2,Unnamed: 20_level_0
0,NaN,Player,Nation,Pos,Age,90s,Att,Live,Dead,FK,...,Crs,TI,CK,In,Out,Str,Cmp,Off,Blocks,Matches
1,0.0,David de Gea,es ESP,GK,31,58.3,1581.0,1265.0,308.0,77.0,...,0,0.0,0.0,0.0,0.0,0.0,1148.0,8.0,1.0,Matches
2,1.0,Bruno Fernandes,pt POR,"MF,FW",27,57.3,2581.0,2330.0,235.0,43.0,...,243,36.0,109.0,26.0,36.0,2.0,1919.0,16.0,82.0,Matches
3,2.0,Marcus Rashford,eng ENG,FW,24,47.6,1020.0,982.0,33.0,3.0,...,38,13.0,0.0,0.0,0.0,0.0,787.0,5.0,38.0,Matches
4,3.0,Casemiro,br BRA,MF,30,43.3,2166.0,2115.0,41.0,36.0,...,24,4.0,0.0,0.0,0.0,0.0,1686.0,10.0,45.0,Matches


### About the dataset (Passing_stats.csv) df2
- **Nation:** 
  - Records in international play at senior level
  - Records in international play at youth level
  - Citizenship presented on Wikipedia
  - Birthplace when available

- **Pos:** Position most commonly played:
  - GK: Goalkeepers
  - DF: Defenders (including FB, LB, RB, CB)
  - MF: Midfielders (including DM, CM, LM, RM, WM, LW, RW, AM)
  - FW: Forwards

- **Age:** Age at the start of the season

- **90s:** 90s played (minutes played divided by 90)

- **Passing Statistics:**
  - Total Passes Completed (Cmp) and Attempted (Att)
  - Pass Completion Percentage (Cmp%)
  - Total Passing Distance (TotDist)
  - Progressive Passing Distance (PrgDist)
  - Short Passes (between 5 and 15 yards)
    - Passes Completed (Cmp)
    - Passes Attempted (Att)
    - Pass Completion Percentage (Cmp%)
  - Medium Passes (between 15 and 30 yards)
    - Passes Completed (Cmp)
    - Passes Attempted (Att)
    - Pass Completion Percentage (Cmp%)
  - Long Passes (longer than 30 yards)
    - Passes Completed (Cmp)
    - Passes Attempted (Att)
    - Pass Completion Percentage (Cmp%)

- **Assists and Expected Assists (xA):**
  - Total Assists (Ast)
  - Expected Assisted Goals (xAG)
  - Expected Assists (xA)
  - Assists minus Expected Goals Assisted (A-xAG)

- **Key Passes (KP):** Passes that directly lead to a shot

- **Passes into Final Third (1/3):** Completed passes that enter the 1/3 of the pitch closest to the goal, not including set pieces

- **Passes into Penalty Area (PPA):** Completed passes into the 18-yard box, not including set pieces

- **Crosses into Penalty Area (CrsPA):** Completed crosses into the 18-yard box, not including set pieces

- **Progressive Passes (PrgP):** Completed passes that move the ball towards the opponent's goal line, excluding passes from the defending 40% of the pitch

In [4]:
df2.head()

,Unnamed: 0,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Total,Total.1,Total.2,Total.3,...,Unnamed: 19_level_0,Unnamed: 20_level_0,Expected,Expected.1,Unnamed: 23_level_0,Unnamed: 24_level_0,Unnamed: 25_level_0,Unnamed: 26_level_0,Unnamed: 27_level_0,Unnamed: 28_level_0
0,NaN,Player,Nation,Pos,Age,90s,Cmp,Att,Cmp%,TotDist,...,Ast,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,PrgP,Matches
1,0.0,David de Gea,es ESP,GK,31,58.3,1148.0,1581.0,72.6,29543.0,...,0,0.1,0.2,-0.1,1.0,9.0,1.0,0.0,0.0,Matches
2,1.0,Bruno Fernandes,pt POR,"MF,FW",27,57.3,1919.0,2581.0,74.4,32978.0,...,13,21.7,14.9,-8.7,150.0,201.0,115.0,15.0,327.0,Matches
3,2.0,Marcus Rashford,eng ENG,FW,24,47.6,787.0,1020.0,77.2,11143.0,...,9,4.3,3.7,4.7,40.0,45.0,63.0,3.0,105.0,Matches
4,3.0,Casemiro,br BRA,MF,30,43.3,1686.0,2166.0,77.8,30359.0,...,6,4.4,4.9,1.6,41.0,214.0,30.0,5.0,232.0,Matches


### Preprocessing the datasets to name the first row as the columns

In [5]:
## real column names are those in the first row
real_column_names = df.iloc[0].values

## renaming the column names
df.columns = real_column_names

##dropping the first row as it consists of the actual column names
df = df.drop(df.index[0])

##Droping the first column as it's name is nan and consists of unrequired values
df=df.drop(columns=df.columns[0])

##we will drop the last column 'Matches' as it does not add any value
# df=df.drop(columns=df.columns[-1]).reset_index()

# df = df.drop(columns=df.columns[0])

df.head()

,Player,Nation,Pos,Age,90s,Att,Live,Dead,FK,TB,...,Crs,TI,CK,In,Out,Str,Cmp,Off,Blocks,Matches
1,David de Gea,es ESP,GK,31,58.3,1581.0,1265.0,308.0,77.0,1.0,...,0,0.0,0.0,0.0,0.0,0.0,1148.0,8.0,1.0,Matches
2,Bruno Fernandes,pt POR,"MF,FW",27,57.3,2581.0,2330.0,235.0,43.0,42.0,...,243,36.0,109.0,26.0,36.0,2.0,1919.0,16.0,82.0,Matches
3,Marcus Rashford,eng ENG,FW,24,47.6,1020.0,982.0,33.0,3.0,6.0,...,38,13.0,0.0,0.0,0.0,0.0,787.0,5.0,38.0,Matches
4,Casemiro,br BRA,MF,30,43.3,2166.0,2115.0,41.0,36.0,18.0,...,24,4.0,0.0,0.0,0.0,0.0,1686.0,10.0,45.0,Matches
5,Luke Shaw,eng ENG,DF,27,40.5,2469.0,2096.0,363.0,44.0,4.0,...,129,238.0,47.0,23.0,10.0,0.0,2048.0,10.0,48.0,Matches


In [6]:
## real column names are those in the first row
real_column_names = df2.iloc[0].values

## renaming the column names
df2.columns = real_column_names

##dropping the first row as it consists of the actual column names
df2 = df2.drop(df2.index[0]).reset_index()

##we will drop the last column 'Matches' as it does not add any value
df2=df2.drop(columns=df2.columns[-1])

##Droping the first column as it's name is nan and consists of unrequired values
df2=df2.drop(columns=df2.columns[:2])


df2.head()

,Player,Nation,Pos,Age,90s,Cmp,Att,Cmp%,TotDist,PrgDist,...,Cmp%,Ast,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,PrgP
0,David de Gea,es ESP,GK,31,58.3,1148.0,1581.0,72.6,29543.0,21024.0,...,44.8,0,0.1,0.2,-0.1,1.0,9.0,1.0,0.0,0.0
1,Bruno Fernandes,pt POR,"MF,FW",27,57.3,1919.0,2581.0,74.4,32978.0,11448.0,...,55.6,13,21.7,14.9,-8.7,150.0,201.0,115.0,15.0,327.0
2,Marcus Rashford,eng ENG,FW,24,47.6,787.0,1020.0,77.2,11143.0,2620.0,...,64.9,9,4.3,3.7,4.7,40.0,45.0,63.0,3.0,105.0
3,Casemiro,br BRA,MF,30,43.3,1686.0,2166.0,77.8,30359.0,11348.0,...,64.1,6,4.4,4.9,1.6,41.0,214.0,30.0,5.0,232.0
4,Luke Shaw,eng ENG,DF,27,40.5,2048.0,2469.0,82.9,32145.0,12375.0,...,52.5,6,4.7,4.2,1.3,43.0,195.0,37.0,15.0,172.0


### Renaming the column names as they are duplicates

In [7]:
df2.columns.values[5] = 'TCmp'
df2.columns.values[6] = 'TAtt'
df2.columns.values[7] = 'TCmp%'
df2.columns.values[10] = 'SCmp'
df2.columns.values[11] = 'SAtt'
df2.columns.values[12] = 'SCmp%'
df2.columns.values[13] = 'MCmp'
df2.columns.values[14] = 'MAtt'
df2.columns.values[15] = 'MCmp%'
df2.columns.values[16] = 'LCmp'
df2.columns.values[17] = 'LAtt'
df2.columns.values[18] = 'LCmp%'

In [8]:
df2.columns

Index(['Player', 'Nation', 'Pos', 'Age', '90s', 'TCmp', 'TAtt', 'TCmp%',
       'TotDist', 'PrgDist', 'SCmp', 'SAtt', 'SCmp%', 'MCmp', 'MAtt', 'MCmp%',
       'LCmp', 'LAtt', 'LCmp%', 'Ast', 'xAG', 'xA', 'A-xAG', 'KP', '1/3',
       'PPA', 'CrsPA', 'PrgP'],
      dtype='object')

In [9]:
## we will convert all the columns except player, nation and pos
columns_to_convert = [col for col in df2.columns if col not in ['Player', 'Nation','Pos']]

df2[columns_to_convert] = df2[columns_to_convert].astype(float)

In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 28 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Player   30 non-null     object 
 1   Nation   30 non-null     object 
 2   Pos      30 non-null     object 
 3   Age      30 non-null     float64
 4   90s      30 non-null     float64
 5   TCmp     27 non-null     float64
 6   TAtt     27 non-null     float64
 7   TCmp%    26 non-null     float64
 8   TotDist  27 non-null     float64
 9   PrgDist  27 non-null     float64
 10  SCmp     27 non-null     float64
 11  SAtt     27 non-null     float64
 12  SCmp%    26 non-null     float64
 13  MCmp     27 non-null     float64
 14  MAtt     27 non-null     float64
 15  MCmp%    26 non-null     float64
 16  LCmp     27 non-null     float64
 17  LAtt     27 non-null     float64
 18  LCmp%    25 non-null     float64
 19  Ast      30 non-null     float64
 20  xAG      27 non-null     float64
 21  xA       27 non-nu

### We will initially be working on the passing dataset (df2)

In [11]:
df2

,Player,Nation,Pos,Age,90s,TCmp,TAtt,TCmp%,TotDist,PrgDist,...,LCmp%,Ast,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,PrgP
0,David de Gea,es ESP,GK,31.0,58.3,1148.0,1581.0,72.6,29543.0,21024.0,...,44.8,0.0,0.1,0.2,-0.1,1.0,9.0,1.0,0.0,0.0
1,Bruno Fernandes,pt POR,"MF,FW",27.0,57.3,1919.0,2581.0,74.4,32978.0,11448.0,...,55.6,13.0,21.7,14.9,-8.7,150.0,201.0,115.0,15.0,327.0
2,Marcus Rashford,eng ENG,FW,24.0,47.6,787.0,1020.0,77.2,11143.0,2620.0,...,64.9,9.0,4.3,3.7,4.7,40.0,45.0,63.0,3.0,105.0
3,Casemiro,br BRA,MF,30.0,43.3,1686.0,2166.0,77.8,30359.0,11348.0,...,64.1,6.0,4.4,4.9,1.6,41.0,214.0,30.0,5.0,232.0
4,Luke Shaw,eng ENG,DF,27.0,40.5,2048.0,2469.0,82.9,32145.0,12375.0,...,52.5,6.0,4.7,4.2,1.3,43.0,195.0,37.0,15.0,172.0
5,Lisandro Martínez,ar ARG,DF,24.0,37.9,1888.0,2169.0,87.0,34120.0,12255.0,...,55.2,0.0,0.8,2.3,-0.8,12.0,155.0,11.0,0.0,128.0
6,Diogo Dalot,pt POR,DF,23.0,36.0,1528.0,1891.0,80.8,27960.0,8472.0,...,59.0,2.0,3.0,2.6,-1.0,41.0,117.0,39.0,14.0,137.0
7,Antony,br BRA,FW,22.0,34.4,864.0,1091.0,79.2,11444.0,2631.0,...,61.8,3.0,4.4,3.9,-1.4,40.0,42.0,48.0,8.0,102.0
8,Christian Eriksen,dk DEN,MF,30.0,34.0,1581.0,1952.0,81.0,26250.0,8459.0,...,54.7,10.0,5.7,4.9,4.3,56.0,211.0,37.0,5.0,224.0
9,Tyrell Malacia,nl NED,DF,22.0,29.9,1132.0,1296.0,87.3,16526.0,4707.0,...,59.6,0.0,1.4,1.6,-1.4,16.0,69.0,20.0,2.0,94.0


### Distribution of the total passes

In [14]:

# Plotting using Plotly Express
fig = px.pie(df2, values='TAtt', names='Player', title='Passes Attempted Distribution by Players')

# Update the layout to set the background color
fig.update_layout(
    plot_bgcolor='black',  # Set the background color to black
    paper_bgcolor='black',  # Set the paper color to black (outside the plot area)
    font=dict(color='white')  # Set the font color to white for better visibility
)

# Show the figure
fig.show()


In [15]:
# Sort the DataFrame based on TCmp% in descending order
df_sorted = df2.sort_values(by='TCmp%', ascending=False)

# Plotting using Plotly Express
fig = px.bar(df_sorted, x='Player', y='TCmp%', title='Pass completion rate')
fig.update_traces(marker_color='red')
fig.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig.show()